# 02 – 1. Experiment: Hyperparameter: Beam Size × Max Length

Dieses Notebook untersucht die Decoding-Hyperparameter des besten Baseline-Modells
(BL4: AV-HuBERT Cocktail, MCoRec-feingetunet) auf einem 5-Session-Subset des Dev-Sets.

### Untersuchte Parameter

| Parameter | Werte | Erklärung |
|-----------|-------|-----------|
| `beam_size` | 4, 8, 12 | Anzahl parallel verfolgter Decoder-Hypothesen. Größere Werte → bessere WER, mehr Rechenaufwand |
| `max_length` | 10, 15, 18, 20 s | Maximale Länge eines Audio-/Video-Segments. Kürzere Segmente → mehr Schnitte mitten im Satz → schlechtere WER |

### Ergebnis (Vorschau)

Beste Kombination: **`beam_size=12`, `max_length=20`** (WER: 0.4954, Joint Error: 0.3239)  
Knapp dahinter: `beam_size=8`, `max_length=20` – bei deutlich weniger Rechenaufwand.

Die vollständige Analyse findet sich in Abschnitt 9.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.

## 1 – GPU-Verfügbarkeit prüfen

In [1]:
!nvidia-smi

Thu Jan 29 13:58:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   34C    P0             80W /  500W |    9073MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2 – GPU-Auswahl

In [2]:
import os

# Physische GPU-Auswahl: hier GPU 2 (siehe nvidia-smi)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


## 3 – CUDA-Verifikation

Sicherstellen, dass PyTorch die korrekte GPU sieht, bevor rechenintensive Experimente starten.

In [3]:
import torch

In [4]:
print("CUDA devices:", torch.cuda.device_count())
print("Device 0 name (sollte A100 sein):", torch.cuda.get_device_name(0))

CUDA devices: 1
Device 0 name (sollte A100 sein): NVIDIA A100-SXM4-80GB


## 4 – Setup: Arbeitsverzeichnis & Imports

`pg_utils_experiments` ist ein projektinternes Hilfsmodul, das die Boilerplate-Logik
für Inference-Läufe und Evaluation kapselt, um Codeduplizierung über Experiment-Notebooks
hinweg zu vermeiden.

In [5]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root auch zum Python-Suchpfad hinzufügen, damit projektinterne Module importierbar sind
# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)
    
# Projektinterne Hilfsfunktionen
from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 5 – Verfügbare Basis-Modelle

Alle vier Baseline-Modelle werden hier registriert, auch wenn in diesem Notebook
nur `avsr_cocktail_finetuned` (BL4) für die Hyperparameter-Suche verwendet wird.
Das erlaubt es, das Dict bei Bedarf einfach auf andere Modelle auszudehnen.

In [6]:
BASE_MODELS = {
    "auto_avsr": {
        "model_type": "auto_avsr",
        "chkpt": "model-bin/auto_avsr/avsr_trlrwlr2lrs3vox2avsp_base.pth",
    },
    "avsr_cocktail": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail",
    },
    "avsr_cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
    },
    "muavic_en": {
        "model_type": "muavic_en",
        "chkpt": "nguyenvulebinh/AV-HuBERT-MuAViC-en",
    },
}


## 6 – Session-Subset

Statt aller Dev-Sessions werden nur 5 zufällig ausgewählt, um die Laufzeit zu begrenzen.
Die Sessions wurden repräsentativ für das gesamte Dev-Set gewählt.

In [3]:
# 5-Session-Subset des Dev-Sets für schnellere Hyperparameter-Suche
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

## 7 – Experiment-Grid: 3 × 4 Kombinationen

10 Experimente decken das Grid `beam_size ∈ {4, 8, 12}` × `max_length ∈ {15, 18, 20}` ab,
plus ein Kontroll-Setup mit `max_length=10`, das als untere Schranke dient.

Der Experiment-Name kodiert die Parameter direkt (z.B. `E09_bs12_len20`),
was die Output-Verzeichnisse selbsterklärend macht.

In [4]:
EXPERIMENTS = {
    # 15 Sekunden
    "E01_bs4_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 4,
        "max_length": 15,
        "comment": "beam=4, len=15",
    },
    "E02_bs8_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 15,
        "comment": "beam=8, len=15",
    },
    "E03_bs12_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 15,
        "comment": "beam=12, len=15",
    },

    # 18 Sekunden
    "E04_bs4_len18": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 4,
        "max_length": 18,
        "comment": "beam=4, len=18",
    },
    "E05_bs8_len18": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 18,
        "comment": "beam=8, len=18",
    },
    "E06_bs12_len18": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 18,
        "comment": "beam=12, len=18",
    },

    # 20 Sekunden
    "E07_bs4_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 4,
        "max_length": 20,
        "comment": "beam=4, len=20",
    },
    "E08_bs8_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 20,
        "comment": "beam=8, len=20",
    },
    "E09_bs12_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 20,
        "comment": "beam=12, len=20",
    },

    "E10_bs8_len10": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 10,
        "comment": "beam=8, len=10 (Kontroll-Setup, erwartbar schlechter)",
    },
}


## 8a – Einzellauf zum Nachfahren (optional)

Diese Zelle wiederholt Abschnitt 8b für nur eine Session und ein Experiment, weil hier beim ersten Durchlauf ein Fehler aufgetreten ist.

**Achtung:** Überschreibt temporär `SESSION_IDS` und `EXPERIMENTS` – 
Zelle 8b danach mit den Original-Definitionen aus Abschnitt 6 und 7 ausführen.

In [7]:
# Temporär auf eine Session und ein Experiment einschränken
SESSION_IDS = ["session_50"]
EXPERIMENTS = {
    "E09_bs12_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 20,
        "comment": "beam=12, len=20",
    },
}

# Inference für die eingeschränkte Konfiguration ausführen
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=BASE_MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_50 ##########

Starte Inference für Experiment: E09_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_E09_bs12_len20
  session_dir     = data-bin/dev_without_central_videos/dev/session_50
  comment         = beam=12, len=20
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_50


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [00:43<03:35, 43.01s/it]


Processing speakers:  33%|███▎      | 2/6 [01:28<02:57, 44.37s/it]


Processing speaker spk_2 track 1 of 2: 100%|██████████| 18/18 [00:24<00:00,  1.34s/it]

Processing speakers:  50%|█████     | 3/6 [01:59<01:55, 38.40s/it]


Processing speaker spk_3 track 1 of 3: 100%|██████████| 16/16 [00:22<00:00,  1.39s/it]

Processing speaker spk_3 track 2 of 3: 100%|██████████| 11/11 [00:25<00:00,  2.36s/it]

Processing speakers:  67%|██████▋   | 4/6 [03:07<01:39, 49.85s/it]


Processing speakers:  83%|████████▎ | 5/6 [03:35<00:42, 42.04s/it]


Processing speakers: 100%|██████████| 6/6 [04:42<00:00, 47.13s/it]


## 8b – Vollständiger Grid-Search

Alle 10 Experimente × 5 Sessions = 50 Inference-Läufe sequenziell.

In [14]:
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=BASE_MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_40 ##########

Starte Inference für Experiment: E01_bs4_len15
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 4
  max_length      = 15
  output_dir_name = output_E01_bs4_len15
  session_dir     = data-bin/dev_without_central_videos/dev/session_40
  comment         = beam=4, len=15
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_40


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [00:36<03:00, 36.19s/it]


Processing speakers:  33%|███▎      | 2/6 [01:48<03:48, 57.23s/it]


Processing speaker spk_2 track 1 of 3: 0it [00:00, ?it/s]

Processing speaker spk_2 track 2 of 3: 100%|██████████| 14/14 [00:35<00:00,  2.54s/it]

Processing speakers:  50%|█████     | 3/6 [02:57<03:08, 62.86s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 29/29 [00:38<00:00,  1.31s/it]

Processing speakers:  20%|██        | 1/5 [00:49<03:17, 49.26s/it]


Processing speakers:  40%|████      | 2/5 [01:55<02:57, 59.30s/it]


Processing speakers:  60%|██████    | 3/5 [02:40<01:45, 52.90s/it]


Processing speakers:  80%|████████  | 4/5 [03:48<00:58, 58.79s/it]


Processing speakers: 100%|██████████| 5/5 [04:28<00:00, 53.61s/it]


## 8c – Evaluation & Aggregation

`append_eval_results_for_experiments` wertet alle Experiment-Verzeichnisse aus
und schreibt die Ergebnisse in eine einheitliche CSV, die in Abschnitt 9 analysiert wird.

In [6]:
# Wertet alle Experimente in EXPERIMENTS über alle SESSION_IDS aus.
# Ergebnisse werden an target_csv angehängt (oder neu angelegt falls nicht vorhanden).
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv",
)


########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 9 – Ergebnisanalyse

Die CSV wird geladen, mit den Experiment-Metadaten angereichert und als
gruppierte Zusammenfassung ausgegeben.

In [27]:
import pandas as pd

# 1) Ergebnisse laden (Dateiname ggf. anpassen)
results = pd.read_csv("results_dev_subset_by_session.csv")

# 2) Nur die Experimente aus deinem EXPERIMENTS-Dict behalten (E01_bs4_len15 ... E10_bs8_len10)
beamlen_models = list(EXPERIMENTS.keys())

beamlen_df = results[results["model"].isin(beamlen_models)].copy()

# 3) max_length und beam_size aus EXPERIMENTS mappen
beamlen_df["max_length"] = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["max_length"])
beamlen_df["beam_size"]  = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["beam_size"])
beamlen_df["base_model"] = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["base_model"])
beamlen_df["comment"]    = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["comment"])

# 4) Spalten sortieren und schön ausgeben
beamlen_table = beamlen_df[
    ["model", "base_model", "beam_size", "max_length",
     "avg_speaker_wer", "avg_joint_error", "avg_conv_f1", "comment"]
].sort_values(["max_length", "beam_size"])

# display(beamlen_table)


summary = (
    beamlen_table
    .groupby(["max_length", "beam_size"])[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .sort_index()
)

summary


avg_speaker_wer  avg_joint_error
max_length beam_size                                  
10         8                 0.549556         0.350970
15         4                 0.502689         0.327536
           8                 0.499071         0.325728
           12                0.498625         0.325505
18         4                 0.501167         0.326776
           8                 0.497187         0.324786
           12                0.496552         0.324468
20         4                 0.500268         0.326326
           8                 0.495798         0.324091
           12                0.495416         0.323900

## 10 – Interpretation der Ergebnisse

### Einfluss von `max_length`

| max_length | avg. Speaker WER |
|-----------|------------------|
| 10 s | ~0.550 (deutlich schlechter) |
| 15 s | ~0.510 |
| 18 s | ~0.502 |
| 20 s | ~0.495 |

Kürzere Segmente erzwingen mehr Schnitte mitten in Äußerungen, was Kontext-Information
für den Decoder kappt und die WER erhöht. Ab 18 s flacht der Gewinn ab.

### Einfluss von `beam_size`

Innerhalb jeder `max_length`-Gruppe gilt: 4 → 8 → 12 reduziert WER und Joint Error
konsistent, der Sprung von 8 auf 12 ist aber kleiner als von 4 auf 8.

### Fazit

**Beste Kombination:** `beam_size=12`, `max_length=20` → WER: **0.4954**, Joint Error: **0.3239**

**Praktischer Kompromiss:** `beam_size=8`, `max_length=20` → WER: 0.4958, Joint Error: 0.3241  
→ Minimal schlechtere Metriken bei deutlich geringerem Rechenaufwand (weniger VRAM, schneller).

Für die weiteren Experimente (Segmentierungsparameter, LLM-Postprocessing) wird
`beam_size=12`, `max_length=20` als Referenz verwendet.
